<h1>Project: Project #2 TSP Genetic Algorithm</h1>
<h4>Course: CAP4630 001</h4>
<h4>Team Members: Adam Clark, Mahmood Sakib, Quang Le</h4>
<h4>Date: 06/10/2023</h4>

<h3>Overview</h3>

<body>
The Traveling Salesman Problem (TSP) is about finding the shortest route that visits a list of cities and returns to the starting city. It's a challenging problem with no known fast solution also known as NP-hard, meaning that there is no known polynomial time solution to the problem.However, there are approximate algorithms like the genetic algorithm that can find solutions close to the optimal one such as Ant Colony or Genetic Algorithm. In this project, we will be using the genetic algorithm to solve the TSP problem. The genetic algorithm is a metaheuristic(high-level problem-solving strategies that guide the exploration and exploitation of search spaces to find near-optimal solutions) inspired by the process of natural selection that belongs to the larger class of evolutionary algorithms. The genetic algorithm repeatedly modifies a population of individual solutions. At each step, the genetic algorithm selects individuals at random from the current population to be parents and uses them to produce the children for the next generation. Over successive generations, the population "evolves" toward an optimal solution. Althought there are many variations of the genetic algorithm, they will boil down to these basic step:

1. Initialization - create an initial population
2. Selection - select parents from the population
3. Crossover - create offspring by mating parents
4. Mutation- mutate offspring
5. reapeat 2-4 until terminatal condition is met
<body>

<h3>How were the cities and distances represented (as a data structure)?</h3>

<body>
<p>
 The cities are represented using a dictionary data structure where the keys represent the city numbers and the values are tuples containing the x and y coordinates of each city. This representation allows easy access to the coordinates of a city for distance calculation. Furthermore the distance are calculated using the distance formula square root of (x2-x1)^2 + (y2-y1)^2 which will be use in our fitness fuction later on.
</p>
</body>

In [2]:
import random
import math

#generate cities
def generate_cities (n=25, width=200, height=200):
  cities= {}
  random.seed(1) #setting the seed to 1 for reproducibility
  for i in range (n): #generating n cities
    x = random.uniform(0, width)  #random x coordinate 0-200
    y = random.uniform(0, height) #random y coordinate 0-200
    cities[i+1] = (int(x),int(y)) #adding the city to the dictionary
  return cities 

#calculate distance between two cities
def calculate_distance (cityA, cityB): 
  xDis = abs(cityA[0] - cityB[0])
  yDis = abs(cityA[1] - cityB[1])
  return (math.sqrt((xDis**2)+(yDis**2))) #distance between two points formula

<h3>How did you encode the solution space?</h3>
<body>
The solution space is encoded as permutations of the cities. Each solution in the population is represented as a list of city numbers, where the order of the numbers determines the sequence of visiting the cities.
</body>
<h3>How did you handle the creation of the initial population?</h3>
<body>
The initial population is created using the <i>create_Initial_Population</i> function. It generates a population of a specified size where each solution is a randomly generated permutation of city numbers. The <i>random.sample</i> function is used to ensure that no city is repeated within a solution.
</body>

In [1]:
#creating the inital population for the genetic algorithm
def create_Initial_Population (size_population,size_cities):
  population = []
  random.seed() 
  for i in range (size_population): #creating a population of n solutions
      population.append(random.sample(range(1, size_cities+1), size_cities))   #creating a solution of n cities but with no duplicates
  return population

<h3>How did you compute the fitness score?</h3>
<body>
the fitness score is computed by calculating the total distance traveled in each solution. The fitness function iterates over each solution in the population. For each solution, it calculates the distance between consecutive cities using the calculate_distance function. If during the iteration get an <i>out of range</i> error it mean that we reach the last city and can calculates the distance from the last city to the inital city we started and sums them up to get the total distance if the last city. The fitness score for each solution is the sum of distances.
</body>

In [2]:
#calculating the fitness of each solution
def fitness (population,cityList):
  fitness_per_solution= [] #list of fitness for per solution
  fitness = [] #list of fitness for each solution
  for i in range (len(population)): #for each solution
    for j in range (len(population[i])):  #for each city in the solution    
      try:
        fitness_per_solution.append(int(calculate_distance(cityList[population[i][j]],cityList[population[i][j+1]]))) #calculating the distance between two cities
      except IndexError:
        fitness_per_solution.append(int(calculate_distance(cityList[population[i][j]],cityList[population[i][0]]))) #calculating the distance between the last city and the first city
    fitness.append(sum(fitness_per_solution)) #summing the distance between all cities in the solution
    fitness_per_solution= [] #list of fitness for per solution
  return fitness  

<h3>Which parent selection strategy did you use? Why?</h3>


<body>
<p>
The parent selection strategy used is a simple random selection from the fittest 50% of the population. The selection function randomly selects two parents from the top 50% of solutions with the lowest fitness (shortest distances). This strategy gives a higher chance for better-performing solutions to be selected as parents, promoting the preservation of good traits in the offspring.
</p>
</body>

In [3]:
#Selecting two parents from the that 50% of the population that have the lowest fitness (distance)
def selection(population,fitness):
  population_fitness = {}
  for i in range (len(population)): #creating a dictionary of the population and their fitness   
    population_fitness[i] = fitness[i]
  fittest_50_percent = [] 
  for i in range (len(population)):
    if population_fitness[i] <= (sum(population_fitness.values())/len(population)): #selecting top 50% of the population
      fittest_50_percent.append(i)
    else:
      pass
  random.seed()#Reset the seeds so we dont get the same value
  parent1= random.choice(fittest_50_percent) #selecting a random parent from the 50% of the population that have the lowest fitness (distance)
  parent2= random.choice(fittest_50_percent) #selecting a random parent from the 50% of the population that have the lowest fitness (distance)
  #validating that the two parents are not the same
  while parent1 == parent2:
    parent2= random.choice(fittest_50_percent) #selecting a random parent from the 50% of the population that have the lowest fitness (distance)
    
  return parent1,parent2

<h3>Which crossover strategy(ies) did you try? Which one worked out best?</h3>

<body>
the original thought was using the single point crossover strategy in which we split the parent genes at a certain point and just do a simple swap which can work for less complex problem like the Knapsack problems or Targeted String Problem but a problem occur in which we will have repeating city in the same solution space thus skewing our fitness function additionally will break our parameter for the problem which dictate that we must travel through all cities. To address the issue of having repeated cities in the same solution space and ensure that all cities are visited, After a bit of research we found (Eric Stoltz) solution which used the Order Crossover (OX) breeding method. This method is specifically designed for permutation-based problems such as the Traveling Salesman Problem (TSP). The function performs a random crossover between two parent solutions. It starts by selecting a random range of genes (city numbers) from one parent. The selected genes are then copied to the offspring in the same positions. The remaining genes are filled in the order they appear in the second parent, while ensuring that no duplicates occur. This way, the order of cities is maintained, and all cities are still visited.

<p>
<i>
Parent 1: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Parent 2: [4, 6, 8, 9, 2, 7, 5, 1, 3]

Generate two random indices, geneA and geneB, within the length of the parent solutions. These indices determine the crossover range.

For example, let's say geneA is 3 and geneB is 7.

Determine the starting and ending points for the crossover range.

startGene is the minimum value between geneA and geneB: 3.
endGene is the maximum value between geneA and geneB: 7.
Create the child solution childP1 by copying the genes within the crossover range from Parent 1.

childP1: [4, 5, 6]
Generate the child solution childP2 by adding the genes from Parent 2 that are not already in childP1. Maintain the order of the genes from Parent 2.

childP2: [8, 9, 2, 7, 1, 3]
Note: The genes 4, 5, and 6 are not added to childP2 since they are already present in childP1.

Combine childP1 and childP2 to create the final child solution.

Child Solution: [4, 5, 6, 8, 9, 2, 7, 1, 3]
</i>
</p>
</body>

In [4]:
#Order cross over breeding
def order_crossover_breed (Parent1,Parent2):
  childP1 = [] #child solution from parent 1
  childP2 = [] #child solution from parent 2
  random.seed() #Reset the seeds so we dont get the same value
  geneA = int(random.random() * len(Parent1)) #creating a random index from parent 1
  geneB = int(random.random() * len(Parent1)) #creating a random index from parent 1
  startGene = min(geneA, geneB) #selecting the least index as the starting point
  endGene = max(geneA, geneB) #selecting the highest index as the ending point
  for i in range(startGene, endGene): #for each gene in the selected range
    childP1.append(Parent1[i]) #adding the gene to the child solution from parent 1
  childP2 = [item for item in Parent2 if item not in childP1] #adding the gene to the child solution from parent 2 that are not already in the child solution from parent 1
  child = childP1 + childP2 #adding the two child solutions together
  return child

<h3>Which mutation strategy(ies) did you try? Which one worked out best?</h3>

The mutation strategy used is a simple swap mutation. The mutation function randomly selects a range of genes within a solution and swaps their positions with a certain probability (mutation rate). This introduces random changes in the solutions and helps explore different areas of the search space.
Population:

Solution 1: [1, 2, 3, 4, 5]

1.Randomly Selecting the Range of Genes: let's assume that geneA = 0 and geneB = 1. This means the selected range for mutation is from the first gene to the second gene.

2.Swapping the Genes: The genes in the selected range are swapped.
  Let's assume that randomSpot = 1 and randomSpot2 = 3. We swap the values at these positions within 
  Solution 1: [1, 4, 3, 2, 5]

3.Returning the Mutated Solution: The mutated solution is returned.
  Solution 1: [1, 4, 3, 2, 5]

In [5]:
def mutation(population, mutation_rate = 0.05):
  geneA = int(random.random() * len(population)) #creating a random index from parent 1
  geneB = int(random.random() * len(population)) #creating a random index from parent 2
  startGene = min(geneA, geneB) #selecting the least index as the starting point
  endGene = max(geneA, geneB) #selecting the highest index as the ending point 
  random.seed()
  if (random.random() < mutation_rate): # Only mutate if the random number is less than mutation rate
    for i in range(startGene, endGene): #for each gene in the selected range
            randomSpot = random.randint(0, len(population[i])-1)
            randomSpot2 = random.randint(0, len(population[i])-1)
            population[i][randomSpot],population[i][randomSpot2] = population[i][randomSpot2],population[i][randomSpot] #swapping the genes
  return population